In [27]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, balanced_accuracy_score

In [7]:
train = pd.read_csv("Train_added_features_final.csv", index_col=0)
test = pd.read_csv("Test_added_features_final.csv", index_col=0)

In [8]:
train.head()

,comp,left,Reused_id,1_ratings,2_ratings,3_ratings,4_ratings,total_num_ratings,lastrating,Delta_days,...,2_reviews_opposed,3_reviews_supported,3_reviews_opposed,4_reviews_supported,4_reviews_opposed,total_opp,total_supp,Reliability,NonReliability,Reliability_factor
id,,,,,,,,,,,,,,,,,,,,,
0,0.066176,0,0.0,0.250000,0.250000,0.250000,0.250000,4.0,4,283,...,0.250000,0.250000,0.250000,0.250000,0.250000,0.0,0.0,15,2,0.882353
4,0.066176,0,0.0,0.000000,0.136364,0.272727,0.590909,22.0,4,92,...,0.000000,0.000000,0.500000,0.666667,0.500000,2.0,3.0,30,8,0.789474
6,0.066176,0,0.0,0.015385,0.025641,0.046154,0.912821,195.0,4,0,...,0.083333,0.212121,0.333333,0.545455,0.416667,12.0,66.0,80,10,0.888889
7,0.066176,0,0.0,0.137931,0.172414,0.448276,0.241379,29.0,4,0,...,0.000000,0.222222,0.000000,0.388889,0.666667,3.0,18.0,69,32,0.683168
8,0.066176,1,1.0,0.066667,0.200000,0.600000,0.133333,15.0,3,2,...,0.083333,0.083333,0.000000,0.500000,0.416667,0.0,12.0,65,72,0.474453


In [30]:
X = train.drop('left', axis=1) 
y = train['left']

In [31]:
np.unique(y.values, return_counts=True)

(array([0, 1], dtype=int64), array([2932,  581], dtype=int64))

#### Due to the presence of class imbalance, negative sampling is done 

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

train_data = pd.DataFrame.copy(X_train)
train_data['left']  = y_train 

In [33]:
# Over Sampling Negative Class

pos_class = train_data[train_data['left']==1]
neg_class = train_data[train_data['left']==0]
neg_resampled = neg_class.sample(n=int(len(pos_class)), replace=False)
    
train_data_resampled = pd.concat([pos_class, neg_resampled])

# Use these for train_dataings

X_train_data_resampled = train_data_resampled.drop('left', axis=1).values
y_train_data_resampled = train_data_resampled['left']

In [37]:
cv_params = {
    'max_depth': -1,
    'objective': 'binary',
    'metric':'auc',  
    'feature_fraction': 0.6, 
    'bagging_fraction': 0.8,
    'reg_lambda': 10,
    'n_estimators':1000
    
    }


lgb_train = lgb.Dataset(X_train_resampled, y_train_resampled)

model = lgb.train(cv_params, 
                lgb_train, 
                verbose_eval=1)

y_train_prob = model.predict(X_train_resampled)
y_test_prob = model.predict(X_test)

     
thres = 0.5
y_pred_train = np.zeros(len(y_train_prob))
y_pred_train[np.argwhere(y_train_prob>thres)] = 1
y_pred_test = np.zeros(len(y_test_prob))
y_pred_test[np.argwhere(y_test_prob>thres)] = 1
     
print('Train Accuracy =', accuracy_score(y_train_resampled, y_pred_train))
print('Train Balanced Accuracy =', balanced_accuracy_score(y_train_resampled, y_pred_train))
print('Train F1 Score =', f1_score(y_train_resampled, y_pred_train))

print('\n\n')

print('Test Accuracy =', accuracy_score(y_test, y_pred_test))
print('Test Balanced Accuracy =', balanced_accuracy_score(y_test, y_pred_test))
print('Test F1 Score =', f1_score(y_test, y_pred_test))
    


Train Accuracy = 0.9982788296041308
Train Balanced Accuracy = 0.9982788296041308
Train F1 Score = 0.9982817869415808



Test Accuracy = 0.883357041251778
Test Balanced Accuracy = 0.9312080536912752
Test F1 Score = 0.722972972972973
